In [1]:
# Week 10: 4/9/18-4/29/18
import pandas as pd
import numpy as np
import datetime
import pandas as pd
from googleads import adwords
import google_analytics_reporting_api as ga
import os

week="_week10" # 2018 Spring

Start_Date_overall='2018-04-09'
End_Data_overall='2018-04-29'

Start_Date_1='2018-04-09'
End_Date_1='2018-04-18'

Start_Date_2='2018-04-19'
End_Date_2='2018-04-29'


today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Saatva/To Lexie/output/running_on_"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

# Google Analytics

In [2]:
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)

    
    return df


In [3]:
# Saatva.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])


ga_data_saatva=ga_data_1.append(ga_data_2)
ga_data_saatva.shape

(3358, 5)

In [4]:
# Saatvamattress.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])

ga_data_2=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions'],
                                            ga_dimensions=['ga:dateHour','ga:sourceMedium'])


ga_data_SM=ga_data_1.append(ga_data_2)
ga_data_SM.shape

(18394, 5)

In [13]:
ga_data_SM=num_func_trans(ga_data_SM)
ga_data_saatva=num_func_trans(ga_data_saatva)
ga_data_both=ga_data_SM.append(ga_data_saatva)

In [14]:
def define_media(df,writing_df_name):
    df['Source']=df['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
    df['Medium']=df['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])
    # msn in 'social'
    # mail.yahoo.com / referral in 'referral'

    df['Defined_Media']=np.nan
    df['Defined_Media']=np.where(df['ga:sourceMedium'].str.contains('youtube.'),"Social",
                                      np.where((df['Source'].isin(['msn','instagram','pinterest','facebook'])) & (df['Medium']=='cpc'),"Social", # Social: 'msn','instagram','pinterest','facebook
                                               np.where(df['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                        np.where(df['Source']=="(direct)","Direct",
                                                                 np.where((df['Source'].isin(['google','bing','yahoo'])) & (df['Medium']=='cpc'),"SEM",
                                                                              np.where((df['Source'].isin(['google','bing','yahoo'])) & (df['Medium']=='organic'),"SEO",
                                                                                       np.where(df['Source']=="powerInbox","Email",
                                                                                                np.where((df['ga:sourceMedium'].str.contains('email')) & (~df['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (df['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                         np.where(df['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                                  np.where(df['Medium']=="referral","Referral",
                                                                                                                           np.where(df['ga:sourceMedium'].str.contains('display'),"Display",
                                                                                                                                    "Others")
                                                                                                                           )
                                                                                                                  )
                                                                                                         )
                                                                                               )
                                                                                      )
                                                                         )
                                                                )
                                                       )
                                              )
                                     )
    GA_taxonomy=df[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()
    GA_taxonomy.reset_index(inplace=True)
    GA_taxonomy=GA_taxonomy.sort_values('Defined_Media')
    del GA_taxonomy['index']
    ga_agg=df.groupby(['ga:dateHour','Defined_Media'])['ga:sessions'].sum().to_frame()
    ga_agg.reset_index(inplace=True)
    ga_agg_wide=ga_agg.pivot('ga:dateHour','Defined_Media','ga:sessions')
    ga_agg_wide.reset_index(inplace=True)
    ga_agg_wide['time_hour']=ga_agg_wide['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))
    writer=pd.ExcelWriter(writer_folder+'/Saatva_'+writing_df_name+'_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+'.xlsx', engine='xlsxwriter')
    ga_agg_wide.to_excel(writer,"reshaped_df_hour_media",index=False)
    df.to_excel(writer,"data",index=False)
    GA_taxonomy.to_excel(writer,"taxonomy",index=False)
    writer.save()     
    media_col=ga_agg_wide.columns.tolist()[1:(len(ga_agg_wide.columns)-1)]
    ga_output_csv=ga_agg_wide.copy()
    ga_output_csv['Date']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[0:8]))
    ga_output_csv['Hour']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[8:10]))
    del ga_output_csv['ga:dateHour']
    ga_output_csv=ga_output_csv.sort_values(['Date','Hour'],ascending=True)
    ga_output_csv=ga_output_csv[['Date','Hour']+media_col]
    ga_output_csv=ga_output_csv.fillna(0)
    ga_output_csv.to_csv(writer_folder+'/Saatva_'+writing_df_name+'_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+'.csv',index=False)
    

In [15]:
define_media(ga_data_SM,"SaatvaMattress_Only")
define_media(ga_data_saatva,"Saatva_Only")
define_media(ga_data_both,"Both")

In [6]:
'''
# Moved into the function
ga_data['Source']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
ga_data['Medium']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])
'''

'\n# Moved into the function\nga_data[\'Source\']=ga_data[\'ga:sourceMedium\'].apply(lambda x: x.split(" / ")[0])\nga_data[\'Medium\']=ga_data[\'ga:sourceMedium\'].apply(lambda x: x.split(" / ")[1])\n'

In [7]:
'''
# Moved into the function
# msn in 'social'
# mail.yahoo.com / referral in 'referral'

ga_data['Defined_Media']=np.nan
ga_data['Defined_Media']=np.where(ga_data['ga:sourceMedium'].str.contains('youtube.'),"Social",
                                  np.where((ga_data['Source'].isin(['msn','instagram','pinterest','facebook'])) & (ga_data['Medium']=='cpc'),"Social", # Social: 'msn','instagram','pinterest','facebook
                                           np.where(ga_data['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                    np.where(ga_data['Source']=="(direct)","Direct",
                                                             np.where((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='cpc'),"SEM",
                                                                          np.where((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='organic'),"SEO",
                                                                                   np.where(ga_data['Source']=="powerInbox","Email",
                                                                                            np.where((ga_data['ga:sourceMedium'].str.contains('email')) & (~ga_data['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (ga_data['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                     np.where(ga_data['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                              np.where(ga_data['Medium']=="referral","Referral",
                                                                                                                       np.where(ga_data['ga:sourceMedium'].str.contains('display'),"Display",
                                                                                                                                "Others")
                                                                                                                       )
                                                                                                              )
                                                                                                     )
                                                                                           )
                                                                                  )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )
'''

'\n# Moved into the function\n# msn in \'social\'\n# mail.yahoo.com / referral in \'referral\'\n\nga_data[\'Defined_Media\']=np.nan\nga_data[\'Defined_Media\']=np.where(ga_data[\'ga:sourceMedium\'].str.contains(\'youtube.\'),"Social",\n                                  np.where((ga_data[\'Source\'].isin([\'msn\',\'instagram\',\'pinterest\',\'facebook\'])) & (ga_data[\'Medium\']==\'cpc\'),"Social", # Social: \'msn\',\'instagram\',\'pinterest\',\'facebook\n                                           np.where(ga_data[\'ga:sourceMedium\'].str.contains(\'criteo\'),"Retarget",\n                                                    np.where(ga_data[\'Source\']=="(direct)","Direct",\n                                                             np.where((ga_data[\'Source\'].isin([\'google\',\'bing\',\'yahoo\'])) & (ga_data[\'Medium\']==\'cpc\'),"SEM",\n                                                                          np.where((ga_data[\'Source\'].isin([\'google\',\'bing\',\'yahoo\'])) & (g

In [8]:
'''
# Moved into the function
GA_taxonomy=ga_data[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()
GA_taxonomy.reset_index(inplace=True)
GA_taxonomy=GA_taxonomy.sort_values('Defined_Media')
del GA_taxonomy['index']
'''

"\n# Moved into the function\nGA_taxonomy=ga_data[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()\nGA_taxonomy.reset_index(inplace=True)\nGA_taxonomy=GA_taxonomy.sort_values('Defined_Media')\ndel GA_taxonomy['index']\n"

In [9]:
'''
# Moved into the function
ga_agg=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:sessions'].sum().to_frame()
ga_agg.reset_index(inplace=True)
'''

"\n# Moved into the function\nga_agg=ga_data.groupby(['ga:dateHour','Defined_Media'])['ga:sessions'].sum().to_frame()\nga_agg.reset_index(inplace=True)\n"

In [10]:
'''
# Moved into the function
ga_agg_wide=ga_agg.pivot('ga:dateHour','Defined_Media','ga:sessions')
ga_agg_wide.reset_index(inplace=True)
'''

"\n# Moved into the function\nga_agg_wide=ga_agg.pivot('ga:dateHour','Defined_Media','ga:sessions')\nga_agg_wide.reset_index(inplace=True)\n"

In [11]:
'''
# Moved into the function
ga_agg_wide['time_hour']=ga_agg_wide['ga:dateHour'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))
'''

'\n# Moved into the function\nga_agg_wide[\'time_hour\']=ga_agg_wide[\'ga:dateHour\'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d%H"))\n'

In [12]:
'''
# Moved into the function
writer=pd.ExcelWriter(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+'.xlsx', engine='xlsxwriter')
'''

"\n# Moved into the function\nwriter=pd.ExcelWriter(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+'.xlsx', engine='xlsxwriter')\n"

In [100]:
'''
# Moved into the function

# Combined both Saatva.com and Saatvamattress.com
ga_agg_wide.to_excel(writer,"reshaped_df_hour_media",index=False)
ga_data.to_excel(writer,"data",index=False)
GA_taxonomy.to_excel(writer,"taxonomy",index=False)
writer.save()
'''

In [101]:
'''
# Moved into the function

media_col=ga_agg_wide.columns.tolist()[1:(len(ga_agg_wide.columns)-1)]
ga_output_csv=ga_agg_wide.copy()
ga_output_csv['Date']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[0:8]))
ga_output_csv['Hour']=ga_output_csv['ga:dateHour'].apply(lambda x:int(x[8:10]))
del ga_output_csv['ga:dateHour']
ga_output_csv=ga_output_csv.sort_values(['Date','Hour'],ascending=True)
ga_output_csv=ga_output_csv[['Date','Hour']+media_col]
ga_output_csv=ga_output_csv.fillna(0)
ga_output_csv.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_1+'_to_'+End_Date_2+week+'.csv',index=False)
'''

# Google AdWords

In [2]:
def get_client(config_path='googleads_saatva.yaml'):
    return adwords.AdWordsClient.LoadFromStorage(config_path)

def download_report_as_stream(client, report_type, fields, start_date, end_date):
    '''Download Adwords report, then save as csv file to base_dir
    Args:
        client: adwords.AdWordsClient.LoadFromStorage()
        report_type: str eg. 'KEYWORDS_PERFORMANCE_REPORT'
        fields: list eg. ["CampaignId", "CampaignName", ... ]
        start_date: str, eg. '2018-02-11'
        end_date: str, eg.
    Return:
        str, path to csv file
    '''
    report_downloader = client.GetReportDownloader(version='v201802')
    # Create report definition.
    report = {
        'reportName': report_type,
        'dateRangeType': 'CUSTOM_DATE',
        'reportType': report_type,
        'downloadFormat': 'CSV',
        'selector': {
            'fields': fields,
            'dateRange': {
                'min': start_date,
                'max': end_date}
        }
    }

    # save output
    return report_downloader.DownloadReportAsStream(report, 
        skip_report_header=True, skip_column_header=False,
        skip_report_summary=True)



In [3]:
client_Saatva = get_client(config_path='/home/jian/Projects/Saatva/To Lexie/googleads_saatva_JL_Saatva.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","Impressions", "Clicks", "Cost"]
response_Saatva = download_report_as_stream(client_Saatva, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Click = pd.read_csv(response_Saatva)
Saatva_Click['Cost']=Saatva_Click['Cost'].apply(lambda x: x/1000000)

In [4]:
client_Saatva_Branding = get_client(config_path='/home/jian/Projects/Saatva/To Lexie/googleads_saatva_JL_Saatva_Branding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","Impressions", "Clicks", "Cost"]
response_Saatva_Branding = download_report_as_stream(client_Saatva_Branding, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Brand_Click = pd.read_csv(response_Saatva_Branding)
Saatva_Brand_Click['Cost']=Saatva_Brand_Click['Cost'].apply(lambda x: x/1000000)

In [5]:
client_Saatva_Competitors = get_client(config_path='/home/jian/Projects/Saatva/To Lexie/googleads_saatva_JL_Saatva_Competitors.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","Impressions", "Clicks", "Cost"]
response_Saatva_Competitor = download_report_as_stream(client_Saatva_Competitors, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Competitor_Click = pd.read_csv(response_Saatva_Competitor)
Saatva_Competitor_Click['Cost']=Saatva_Competitor_Click['Cost'].apply(lambda x: x/1000000)

In [6]:
Saatva_Click=Saatva_Click[~((Saatva_Click['Campaign'].str.contains("Loom")) | (Saatva_Click['Campaign'].str.contains("Leaf")) | (Saatva_Click['Campaign'].str.contains("Zenhaven")))]

Saatva_Competitor_Click=Saatva_Competitor_Click[Saatva_Competitor_Click['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Saatva_Competitor_Click.reset_index(inplace=True)
del Saatva_Competitor_Click['index']

Saatva_Click['Campaign_Type']=Saatva_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Saatva_Brand_Click['Campaign_Type']=Saatva_Brand_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM_Brand").replace("Shopping","PLA"))
Saatva_Competitor_Click['Campaign_Type']="SEM_Non_Brand" # Because it's all labeled as Non-Brand in Campaign and in Search Only Campaign Type

In [7]:
# Split Saatva
# 'Search Network with Display Select' Campaign Type all 0 Impr and Clicks
Saatva_Click_Search=Saatva_Click[Saatva_Click['Advertising Channel']=='Search']
Saatva_Click_Others=Saatva_Click[Saatva_Click['Advertising Channel']!='Search']

In [8]:
Saatva_Click_Search['Campaign_Type']=np.where((Saatva_Click_Search['Campaign'].str.contains("Branded")) | (Saatva_Click_Search['Campaign'].str.contains(" Brand ")) | (Saatva_Click_Search['Campaign'].str.contains("Co-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Saatva_Click_Search['Campaign'].str.contains("Non-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Saatva_Click_Search_1=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']!="NA"]
Saatva_Click_Search_2=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']=="NA"]
len(Saatva_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0

In [9]:
# All campaing is able to differentiate from the name about brand and non-brand, so no need to go to keywords level
'''
Saatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)
Saatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva branding_keywords.csv",dtype=str)
Saatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)

Saatva_keywords=Saatva_keywords[~((Saatva_keywords['Campaign'].str.contains("Loom")) | (Saatva_keywords['Campaign'].str.contains("Leaf")) | (Saatva_keywords['Campaign'].str.contains("Zenhaven")))]
Saatva_keywords['Campaign_Type']=np.where(Saatva_keywords['Search keyword'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].astype(int)
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].astype(int)
Saatva_keywords=Saatva_keywords.groupby(['Campaign','Campaign_Type'])['Impressions','Clicks'].sum()
Saatva_keywords.reset_index(inplace=True)
Saatva_keywords=Saatva_keywords.sort_values(['Campaign','Impressions'])
Saatva_keywords=Saatva_keywords.drop_duplicates('Campaign')
Saatva_Campaign_Type=Saatva_keywords[['Campaign','Campaign_Type']]

del Saatva_Click_Search_2['Campaign_Type']
Saatva_Click_Search_2=pd.merge(Saatva_Click_Search_2,Saatva_Campaign_Type,on="Campaign",how="left")

Saatva_Search_Click=Saatva_Click_Search_1.append(Saatva_Click_Search_2)
Saatva_Search_Click['Campaign_Type']=Saatva_Search_Click['Campaign_Type'].fillna("All_0_Values_and_No_Kws")
'''

'\nSaatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)\nSaatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva branding_keywords.csv",dtype=str)\nSaatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To Lexie/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)\n\nSaatva_keywords=Saatva_keywords[~((Saatva_keywords[\'Campaign\'].str.contains("Loom")) | (Saatva_keywords[\'Campaign\'].str.contains("Leaf")) | (Saatva_keywords[\'Campaign\'].str.contains("Zenhaven")))]\nSaatva_keywords[\'Campaign_Type\']=np.where(Saatva_keywords[\'Search keyword\'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].apply(lambda x: int(x.replace(",","")))\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].astype(int)\nSaatva_keywords[\'Clicks\']=Saatva_keywords[\'Clicks\'].apply(lambda x: int(x.replace(",","")))

In [10]:
Saatva_Click=Saatva_Click_Search.append(Saatva_Click_Others)

In [11]:
writer=pd.ExcelWriter(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+".xlsx",engine='xlsxwriter')

In [12]:
Saatva_Agg=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click)
Saatva_Agg['Day']=Saatva_Agg['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Saatva_Agg.reset_index(inplace=True)
del Saatva_Agg['index']
Saatva_Agg['Hour of day']=Saatva_Agg['Hour of day'].astype(int)
Saatva_Agg_Category=Saatva_Agg[['Campaign','Account','Advertising Channel','Campaign_Type']].drop_duplicates()
Saatva_Agg_Category.to_excel(writer,"Campaign Category",index=False)

In [13]:
Saatva_Agg=Saatva_Agg.groupby(["Campaign_Type","Day","Hour of day"])['Impressions','Clicks','Cost'].sum()
Saatva_Agg.reset_index(inplace=True)
Saatva_Agg['CTR']=Saatva_Agg['Clicks']/Saatva_Agg['Impressions']
Saatva_Agg.to_excel(writer,"reshaped by hour day",index=False)

Saatva_Agg_No_NA=Saatva_Agg[(Saatva_Agg['Campaign_Type']!="NA") & (Saatva_Agg['Campaign_Type']!="Search Network with Display Select")]

Saatva_Agg_No_NA.to_excel(writer,"clean data by hour day",index=False)

In [14]:
writer.save()

In [15]:
AdWords_output=Saatva_Agg.copy()
AdWords_output['Date']=AdWords_output['Day'].apply(lambda x: str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
AdWords_output=AdWords_output[['Campaign_Type','Date','Hour of day','Impressions','Clicks','Cost']]
AdWords_output=AdWords_output.rename(columns={"Hour of day":"Hour","Impressions":"IMP"})
AdWords_output=AdWords_output.sort_values(["Campaign_Type","Date","Hour"],ascending=[True,True,True])
AdWords_output.to_csv(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+".csv",index=False)

,Campaign_Type,Date,Hour,IMP,Clicks
2515,Video,20180429,19,10896,36
2516,Video,20180429,20,11352,45
2517,Video,20180429,21,12303,52
2518,Video,20180429,22,12690,29
2519,Video,20180429,23,6925,24
